# Passivation of Silicon Nanowire

## Setup the environment

In [ ]:
# Enable interactive selection of coordination threshold
IS_COORDINATION_SELECTION_INTERACTIVE = False

BOND_LENGTH = 1.46
PASSIVANT = "H"
COORDINATION_SEARCH_RADIUS = 2.5
COORDINATION_THRESHOLD = 3
MAX_BONDS_TO_SATURATE = 2

## Load the Silicon material

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

silicon = Material(Materials.get_by_name_first_match('Silicon'))

## Visualize the Silicon material

In [ ]:
from utils.visualize import visualize_materials
visualize_materials(silicon)

## Create a silicon nanowire

In [ ]:
from mat3ra.made.tools.modify import rotate, filter_by_box, add_vacuum_sides
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration

slab_config = SlabConfiguration(bulk=silicon, miller_indices=(1,1,0), thickness=2, vacuum=0, use_orthogonal_z=True, xy_supercell_matrix=[[3,0],[0,2]]) 
silicon_slab = create_slab(slab_config)
rotated_slab = rotate(silicon_slab, "z", -45)
silicon_nanowire = filter_by_box(rotated_slab, [0.2,0.28,0], [0.78,0.87,1])
nanowire_xy_pbc = rotate(silicon_nanowire, "z", -45)
nanowire_xy_pbc = add_vacuum_sides(nanowire_xy_pbc, 10.0, on_x=True, on_y=True)
visualize_materials(nanowire_xy_pbc)
visualize_materials(nanowire_xy_pbc, rotation="-90x")

## Compute the radial distribution function (RDF) of the Material

In [ ]:
from mat3ra.made.tools.convert import to_pymatgen
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist

structure = to_pymatgen(nanowire_xy_pbc)

# Parameters for RDF
cutoff = 10.0  # Maximum distance for RDF calculation
bin_size = 0.1  # Size of each bin in the histogram

# Get the Cartesian coordinates of all atoms
coords = structure.cart_coords

# Compute pairwise distances
distances = pdist(coords)  # Pairwise distances
distances = distances[distances <= cutoff]  # Only consider distances within the cutoff

# Bin distances into a histogram
bins = np.arange(0, cutoff + bin_size, bin_size)  # Bin edges
hist, bin_edges = np.histogram(distances, bins=bins, density=False)

# Convert to radial distribution function
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
shell_volumes = (4 / 3) * np.pi * (np.power(bin_edges[1:], 3) - np.power(bin_edges[:-1], 3))  # Volume of spherical shells
density = len(coords) / structure.volume  # Atomic density (number of atoms per unit volume)
rdf = hist / (shell_volumes * density)  # Normalize histogram

# Plot the RDF
plt.figure(figsize=(8, 5))
plt.plot(bin_centers, rdf, label="Radial Distribution Function")
plt.xlabel("Distance (Å)")
plt.ylabel("g(r)")
plt.title("Radial Distribution Function (RDF)")
plt.legend()
plt.grid()
plt.show()

## Set the passivation parameters including the coordination threshold

In [ ]:
from utils.io import select_coordination_threshold
from mat3ra.made.tools.build.passivation import get_unique_coordination_numbers, PassivationConfiguration
from mat3ra.made.tools.build.passivation.builders import CoordinationBasedPassivationBuilder, \
    CoordinationBasedPassivationBuilderParameters

config = PassivationConfiguration(
    slab=nanowire_xy_pbc,
    passivant=PASSIVANT,
    bond_length=BOND_LENGTH
)
builder_params = CoordinationBasedPassivationBuilderParameters(
    shadowing_radius=COORDINATION_SEARCH_RADIUS,
    coordination_threshold=0,
)
passivation_builder = CoordinationBasedPassivationBuilder(builder_params)


coordination_numbers     = sorted(list(passivation_builder.get_unique_coordination_numbers(nanowire_xy_pbc)))
print(f"Unique coordination numbers: {coordination_numbers}")
coordination_numbers = sorted(list(get_unique_coordination_numbers(config, builder_params)))
print(f"Unique coordination numbers: {coordination_numbers}")

coordination_threshold = COORDINATION_THRESHOLD
if IS_COORDINATION_SELECTION_INTERACTIVE:
    coordination_threshold = await select_coordination_threshold(coordination_numbers, COORDINATION_THRESHOLD)

## Passivate the Material

In [ ]:
from mat3ra.made.tools.build.passivation import CoordinationBasedPassivationBuilderParameters, CoordinationBasedPassivationBuilder, create_passivation
params = CoordinationBasedPassivationBuilderParameters(
    shadowing_radius=COORDINATION_SEARCH_RADIUS, 
    coordination_threshold=coordination_threshold,    
    bonds_to_passivate=MAX_BONDS_TO_SATURATE
)

passivation_builder = CoordinationBasedPassivationBuilder(params)
passivated_nanowire = create_passivation(config, passivation_builder)


## Visualize the resulting Material

In [ ]:
visualize_materials([{"material": passivated_nanowire}, {"material": passivated_nanowire, "rotation": "-90x"}])

## Save the Material

In [ ]:
from utils.jupyterlite import set_materials
passivated_nanowire.name = "H-Passivated Silicon Nanowire"
set_materials([passivated_nanowire])